In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_recipes = pd.read_csv("recipes_imputed_manual.csv")

In [9]:
# TODO:
# here I could also categorise the prep time into a smaller set of categories to improve overlap and reduce sparsity.



df_dummy = pd.get_dummies(df_recipes.drop("recipe_id", axis=1))

In [13]:

array_cos_hard = cosine_similarity(df_dummy, df_dummy)

In [19]:
print(f"{len(array_cos_hard)}, {len(array_cos_hard[0])}")

260, 260


In [25]:
df_similarity = pd.DataFrame(
        data=array_cos_hard,
        columns=df_recipes.recipe_id.values,
        index=df_recipes.recipe_id.values
    )

In [35]:
def get_top_product_recs(recs_mat, nor, product_id_mapping):
    """
    Compute the recipe-recipe top similarity array.

    Args:
        recs_mat (matrix array): containing the product-product similarity information.
        nor (int): The number of recommendations to produce for each product.
        product_id_mapping (numpy vectorised function): A function that maps an integer back to a product id

    Return:
        recommendations (array): Array containing the resulting recommendations for each product
    """
    recs_mat_array = np.array(recs_mat)
    recs_data = []

    for count, row in enumerate(recs_mat_array):
        recs = np.argsort(-row)[: nor + 1]
        recs = recs[recs != count]
        recs = np.insert(recs, 0, count)[: nor + 1]
        recs_data.append(recs)

    out = np.array(recs_data)
    recommendations = product_id_mapping(out)
    return np.array(recommendations)

In [37]:
recipes_id_dict = dict(zip(range(df_similarity.shape[0]), df_similarity.index))
product_id_mapping = np.vectorize(lambda x: recipes_id_dict.get(x))

recs = get_top_product_recs(
        recs_mat=df_similarity,
        nor=5,
        product_id_mapping=product_id_mapping
    )

In [40]:
df_recs = pd.DataFrame(
        data=recs,
        columns=list(
            ["recipe", "rec_1", "rec_2", "rec_3",
             "rec_4", "rec_5"]
        ),
    )

In [56]:
# This method clearly allows for vegan/meat/vegetarian recipes to be cross-recommended, but that 
# can easily be forced using some clever hard segmentation of the initial data set

from IPython.display import display


for index, row in df_recs.iterrows():
    print(f"*" * 100)
    print(f"Recs for: {row['recipe']}")
    display(df_recipes[df_recipes['recipe_id']==row['recipe']])
        
    print(f"-" * 100)
          
    display(df_recipes[df_recipes['recipe_id'].isin(row[1:].values)])
    print(f"*" * 100)
    

****************************************************************************************************
Recs for: 2


,recipe_id,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
0,2,India,India,Stove top / bowl food,Curry,Fish,Basmati,Rice,Shellfish,Prawns,Fish & Seafood,No,No Spice,25


----------------------------------------------------------------------------------------------------


,recipe_id,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
123,470,India,India,Stove top / bowl food,Curry,Vegetarian,Basmati,Rice,Tofu,Protein not found,Vegetarian,No,No Spice,30
125,473,India,India,Stove top / bowl food,Curry,Fish,Basmati,Rice,Oily Fish,Salmon,Fish & Seafood,No,No Spice,30
127,477,Spain,Spain,Stove top / bowl food,Pilaf / rice,Fish,Basmati,Rice,Shellfish,Prawns,Fish & Seafood,No,No Spice,35
157,555,India,India,Stove top / bowl food,Curry,Meat,Basmati,Rice,Chicken,Breast Chicken,Poultry & Meat,No,No Spice,30
176,882,India,India,Stove top / bowl food,Curry,Fish,Basmati,Rice,Smoked Fish,Smoked Basa,Fish & Seafood,No,Mild,20


****************************************************************************************************
****************************************************************************************************
Recs for: 5


...

****************************************************************************************************
****************************************************************************************************
Recs for: 1284


,recipe_id,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
259,1284,Spain,Spain,Stove top / bowl food,Stew,Meat,Baguette,Bread,Beans,Chickpeas,Pulses,No,Spicy,30


----------------------------------------------------------------------------------------------------


,recipe_id,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
60,330,Spain,Spain,Protein&Veg,Egg dish,Vegetarian,white potatoes,Potato,Beans,Cannellini beans,Pulses,No,Spicy,30
223,1185,Morocco,Morocco,Protein&Veg,Meat & Side veg,Meat,Pitta,Bread,Beans,Chickpeas,Pulses,No,No Spice,30
225,1188,India,India,Stove top / bowl food,Curry,Vegetarian,Naan,Bread,Beans,Chickpeas,Pulses,No,No Spice,30
242,1242,France,France,Stove top / bowl food,Stew,Meat,Baguette,Bread,Chicken,Breast Chicken,Poultry & Meat,No,No Spice,20
248,1251,Italy,Italy,Stove top / bowl food,Stew,Meat,Baguette,Bread,Lentils,Canned lentils,Pulses,No,No Spice,40


****************************************************************************************************


In [24]:
df_similarity

,2,5,9,16,19,20,30,53,54,57,...,1270,1276,1277,1278,1279,1280,1281,1282,1283,1284
2,1.000000,0.076923,0.461538,0.307692,0.153846,0.153846,0.230769,0.153846,0.307692,0.076923,...,0.153846,0.153846,0.153846,0.461538,0.230769,0.153846,0.461538,0.076923,0.384615,0.153846
5,0.076923,1.000000,0.076923,0.153846,0.153846,0.153846,0.230769,0.076923,0.230769,0.153846,...,0.153846,0.230769,0.307692,0.076923,0.076923,0.153846,0.307692,0.000000,0.230769,0.307692
9,0.461538,0.076923,1.000000,0.076923,0.076923,0.076923,0.153846,0.076923,0.230769,0.000000,...,0.076923,0.076923,0.076923,0.076923,0.153846,0.076923,0.153846,0.000000,0.230769,0.230769
16,0.307692,0.153846,0.076923,1.000000,0.230769,0.230769,0.153846,0.153846,0.153846,0.076923,...,0.461538,0.230769,0.153846,0.538462,0.307692,0.153846,0.307692,0.076923,0.230769,0.076923
19,0.153846,0.153846,0.076923,0.230769,1.000000,0.615385,0.307692,0.461538,0.307692,0.076923,...,0.230769,0.307692,0.153846,0.230769,0.076923,0.153846,0.076923,0.307692,0.230769,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0.153846,0.153846,0.076923,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.230769,...,0.307692,0.384615,0.615385,0.153846,0.076923,1.000000,0.076923,0.076923,0.153846,0.230769
1281,0.461538,0.307692,0.153846,0.307692,0.076923,0.076923,0.538462,0.076923,0.307692,0.000000,...,0.076923,0.153846,0.230769,0.384615,0.307692,0.076923,1.000000,0.153846,0.384615,0.153846
1282,0.076923,0.000000,0.000000,0.076923,0.307692,0.307692,0.538462,0.384615,0.384615,0.153846,...,0.230769,0.307692,0.076923,0.153846,0.000000,0.076923,0.153846,1.000000,0.307692,0.076923
1283,0.384615,0.230769,0.230769,0.230769,0.230769,0.307692,0.615385,0.307692,0.846154,0.076923,...,0.153846,0.384615,0.307692,0.230769,0.153846,0.153846,0.384615,0.307692,1.000000,0.153846
